<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Patient Satisfaction Analytics using Large Language Models and Enteprise Vector Store
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style="font-size:20px;font-family:Arial"><b>Introduction:</b></p>

<b style = 'font-size:18px;font-family:Arial'>What is Patient Satisfaction?</b>
<p style = 'font-size:16px;font-family:Arial'>Patient satisfaction measures how happy a patient is with their healthcare experience, reflecting whether their expectations were met. It's a key indicator of quality and impacts various aspects of healthcare, including clinical outcomes, patient retention, and legal claims. Effective communication, efficient processes, and positive interactions all contribute to higher patient satisfaction.</p>

<b style = 'font-size:18px;font-family:Arial'>Why is Patient Satisfaction so important?</b>

<p style = 'font-size:16px;font-family:Arial'>Patient satisfaction is crucial in healthcare because it's a key indicator of both the quality of care and the overall patient experience. Satisfied patients are more likely to adhere to treatment plans, have better health outcomes, and recommend their healthcare providers. Additionally, patient satisfaction can influence factors like patient retention, loyalty, and the financial well-being of healthcare organizations. And provider reimbursement in healthcare is increasingly linked to patient satisfaction through higher reimbursement rates.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Challenges with operationalizing AI</b></p>

<p style = 'font-size:16px;font-family:Arial'>Healthcare organizations of all types are struggling with how to best leverage the promise and the power of new Large Language Model and other AI models, tools, and techniques.  Some of the top challenges include:</p>

<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Privacy and confidentiality</b>.  Many model services require users to send their data and/or prompts to third-party systems over the open internet. Furthermore, models have not been trained or augmented with any non-public data; and therefore may not even be able to provide any actionable results</li>
    <li><b>Actionable insights</b>.  AI tools provide unprecedented capabilities in extracting <b>meaning</b> from unstructured data (text, images, audio, etc.).  Insight alone isn't enough - organizations must contextualize this meaning with existing analytics and data</li>
    <li><b>Scale</b>.  Value increases with automation and scale.  Processing and analyzing millions or billions of records, providing analytics to users, applications, and customers in real-time are both critical requirements for deriving true business value from AI</li>
    </ul>

<b style = 'font-size:18px;font-family:Arial'>Demonstrations</b>
<hr>

<table style = 'width:100%;table-layout:fixed;font-family:Arial'>
    <tr><td style = 'vertical-align:top' width = '25%'><p style = 'font-size:20px;font-family:Arial'><b>Patient Sentiment Analysis</b>
            <ol style = 'font-size:16px;font-family:Arial'>
                <li>Connect and inspect call center transcripts stored in Vantage, object storage, or third-party systems</li>
                <br>      
                <li>Extract sentiment using native ClearScape Analytics functions accessing external LLMs</li>
                <br>
                <li>Analyze results using native analytics functions</li>
            </ol>
        </td><td><img src = 'images/Sentiment_Analysis_AWS.png' width = '100%'></td></tr>
    </table>
    <hr>
<table style = 'width:100%;table-layout:fixed;font-family:Arial'>
    <tr><td style = 'vertical-align:top' width = '25%'><p style = 'font-size:20px;font-family:Arial'><b>Semantic Search for Targeted Patient Outreach</b>
            <ol style = 'font-size:16px;font-family:Arial'>
                <li>Connect and inspect call center transcripts stored in Vantage, object storage, or third-party systems</li>
                <br>      
                <li>Generate embeddings based on the semantic meaning of the call-center transcripts</li>
                <br>
                <li>Identify patients for targeted outreach using built-in analytics and RAG capabilities</li>
            </ol>
        </td><td><img src = 'images/Semantic_Search_AWS.png' width = '100%'></td></tr>
</table>

<hr style='height:2px;border:none'>
<b style = 'font-size:20px;font-family:Arial'>1. Configure the environment</b>

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
!pip install --force-reinstall pillow ipywidgets --quiet
!pip install wordcloud nltk --upgrade --force-reinstall --quiet
!pip install --force-reinstall pillow --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Please</b><i> restart the kernel after executing the above cell to include/update these libraries into memory for this kernel. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i> and then clicking <b>Restart</b>.</p>
</div>

In [ ]:
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi
from teradatasqlalchemy.types import *
import pandas as pd
import csv, sys, os, warnings, getpass
# from collections import OrderedDict
import ipywidgets as widgets
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import plotly.graph_objects as go

from IPython.display import clear_output , display as ipydisplay, Markdown
import matplotlib.pyplot as plt

# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>2. Connect to VantageCloud Lake</b></p>
<p style = 'font-size:16px;font-family:Arial'>Connect to VantageCloud using `create_context` from the teradataml Python library. Input your connection details, including the host, username, password and Analytic Compute Group name.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=Patient_Satisfaction_Analyzer_EVS_VCL.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<hr style="height:2px;border:none;">
<p style="font-size:20px;font-family:Arial"><b>3. Authenticate into User Environment Service (UES) for Container Management</b></p>

<p style="font-size:16px;font-family:Arial"><b>UES authentication</b> is required to create and manage the Python or R environments that we will be creating.  A VantageCloud Lake user can easily create the authentication objects using the Console in a VantageCloud Lake environment.  The step to create these authentication objects has already been performed for you.
</p>
<p style="font-size:16px;font-family:Arial">
   
<ul style="font-size:16px;font-family:Arial; margin-top:4px;">
  <li><a href="https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Teradata-Package-for-Python-on-VantageCloud-Lake/Working-with-Open-Analytics/APIs-to-Use-with-Open-Analytics-Framework/API-to-Set-Authentication-Token/set_auth_token">Click here</a> to see more details about using the Teradata APIs to set the authentication objects.</li>

  <li>Check out <a href="https://medium.com/teradata/deploy-hugging-face-llms-on-teradata-vantagecloud-lake-with-nvidia-gpu-acceleration-d94d999edaa5">Step 4</a> of this tutorial to to see more details about configuring a VantageCloud Lake Environment to use our Open Analytics Framework</li>
</ul>

In [ ]:
# We've already loaded all the values into our environment variables and into a dictionary, env_vars.
# username=env_vars.get("username") isn't required when using base_url, pat and pem.
ues_uri=env_vars.get("ues_uri")
base_uri=ues_uri[:-15]
if set_auth_token(base_url=ues_uri,
                  pat_token=env_vars.get("access_token"), 
                  pem_file=env_vars.get("pem_file")
                 ):
    print("UES Authentication successful")
else:
    print("UES Authentication failed. Check credentials.")
    sys.exit(1)

<hr style="height:2px;border:none;">
<p style="font-size:20px;font-family:Arial"><b>4. Sentiment Analysis</b></p>

<p style = 'font-size:16px;font-family:Arial'>Extract the emotion (anger, joy, sadness, love) and strength of that emotion (0-1) based on analysis of unstructured call center transcripts.  Inspect the distribution of how strongly patients are feeling these emotions - are we reaching a point where we need to change business practices or improve outreach?</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial'>
    <tr><td style = 'vertical-align:top' width = '25%'>
            <ol style = 'font-size:16px;font-family:Arial'>
                <li>Connect and inspect call center transcripts stored in Vantage, object storage, or third-party systems</li>
                <br>      
                <li>Extract sentiment using native ClearScape Analytics functions accessing external LLMs</li>
                <br>
                <li>Analyze results using native analytics functions</li>
            </ol>
        </td><td><img src = 'images/Sentiment_Analysis_AWS.png' width = '100%'></td></tr>
    </table>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Load the Data</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo in the lake environment. The data is available in the database "DEMO_SalesForecasting". Your user should have read access to the database. In case of any issues please write a mail to the support group ("SC230208@teradata.com").</p>

<p style = 'font-size:16px;font-family:Arial'>**Note: The tables are available in DEMO_PatientSatisfaction_DB database and we have created views in DEMO_PatientSatisfaction database which are used in the cells below.</p>


In [ ]:
tdf_transcripts = DataFrame(in_schema("DEMO_PatientSatisfaction","patient_feedback"))
tdf_transcripts


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>6. Sentiment Extraction</b></p>


<p style = 'font-size:18px;font-family:Arial'><b>6.1 Set up Sentiment Extraction Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use the AWS Bedrock implementation of Anthropic claude-v2 for a balance of price and performance.  Enter a valid key/secret/region that has this model enabled, or use an auth object.</p>

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Configuring AWS CLI</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will prompt us for the following information:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>


In [ ]:
access_key = input('AWS Access Key <enter for none>: ')
secret_key = getpass.getpass('AWS Secret Key <enter for none>: ')
auth = input('Auth object <enter for none>:')
region = input('Region: ')

In [ ]:
# Provide model details
# anthropic.claude-3-5-sonnet-20240620-v1:0
# anthropic.claude-3-5-sonnet-20241022-v2:0
# anthropic.claude-3-haiku-20240307-v1:0
# anthropic.claude-3-5-haiku-20241022-v1:0
model_name = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0' #'anthropic.claude-v2'#'anthropic.claude-v2'#'anthropic.claude-instant-v1'
# 'anthropic.claude-v2'
# 'anthropic.claude-instant-v1'

if access_key == '': # if using an auth object
    
    # Select in-database or external model
    llm = TeradataAI(api_type = 'AWS',
                     model_name = model_name,
                     region = region,
                     authorization = auth)
else:
    # Select in-database or external model
    llm = TeradataAI(api_type = 'AWS',
                     model_name = model_name,
                     region = region,
                     access_key = access_key,
                     secret_key = secret_key)


obj = TextAnalyticsAI(llm=llm)


<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>6.2 Extract Patient Sentiment</b></p>

<p style = 'font-size:16px;font-family:Arial'>A simple method call will extract the sentiment for patient comments <b>in-database</b> using the desired LLM and CSP provider.</p>

In [ ]:
tdf_transcripts_5=tdf_transcripts.iloc[0:200]
tdf_transcripts_5.shape

In [ ]:
tdf_transcripts

In [ ]:
%%time
tdf_sentiment = obj.analyze_sentiment(column='Transcript', data=tdf_transcripts_5)
tdf_sentiment


<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>6.3 Execute simple descriptive analytics</b></p>

<p style = 'font-size:16px;font-family:Arial'>Now, we can begin asking increasingly more complex questions of the data; what's the distribution of sentiment?  Or - does any individual patient have increasing feelings of anger, fear, or sadness?  Native, in-database <b>ClearScape Analytics</b> functions can answer these questions at extreme speed and scale.</p>

In [ ]:
CategoricalSummary(data = tdf_sentiment, target_columns = 'Sentiment').result

In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook' 
df = CategoricalSummary(data = tdf_sentiment, target_columns = 'Sentiment').result.to_pandas()
fig = px.bar(df, x = 'DistinctValue', y = 'DistinctValueCount')
fig.show()


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>7. Develop targeted patient outreach with Semantic Search</b></p>


<p style = 'font-size:16px;font-family:Arial'>Match our patients with <b>targeted</b> outreach programs based on the <b>Semantic Meaning</b> of their comments history.  This can allow our organization to provide tailored programs or assistance based on the intent of the commentary, allowing for more accurate targeting.</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial'>
    <tr><td style = 'vertical-align:top' width = '25%'>
            <ol style = 'font-size:16px;font-family:Arial'>
                <li>Create a Vector Store using CSP LLM services and store data in Vantage</li>
                <br>      
                <li>Perform interactive analysis and generate recommended responses to customer complaints</li>
                <br>
                <li>Identify patients for targeted outreach using similarity search functions</li>
            </ol>
        </td><td><img src = 'images/Semantic_Search_AWS.png' width = '100%'></td></tr>
</table>



<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>7.1 Enterprise Vector Store</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>Vector Embedding</b> is a numerical representation of data that captures semantic relationships and similarities, making it possible to perform mathematical operations and comparisons on the data for various tasks like text analysis and recommendation systems.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Generative Responses</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>Use Natural Language</b> to inspect the patient complaints data, and leverage <b>Private LLMs</b> to generate rich, actionable responses.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Hyper-personalize</b> the appropriate action based on AI-based semantic analysis and search.</p>

In [ ]:
vs = VectorStore(name = 'Transcript_Analysis')

In [ ]:
# if desired, destroy and recreate the vector store - or to use an existing one
# skip to the "similarity_search" code below
# vs.destroy()

In [ ]:
# Python
vs.create(search_algorithm = 'VECTORDISTANCE',
          object_names = DataFrame(in_schema("DEMO_PatientSatisfaction","patient_detail_feedback")),
          key_columns = ['Call_ID', 'first_name', 'last_name'],
          data_columns = ['Transcript'],
          vector_column = 'Embedding')

In [ ]:
vs.status()

In [ ]:
vs.similarity_search(question = 'top 10 most angry unique callers').similar_objects[['score','Call_ID','Transcript']]

In [ ]:
from IPython.display import Markdown
Markdown(vs.ask(question = 'the top angry customers', prompt = 'suggest a targeted outreach program'))


<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>7.2 Construct a list of patient care programs</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a small list of topics that will represent the outreach programs we want to target.  This could represent existing outreach or marketing programs we want to more accurately target.

<p style = 'font-size:16px;font-family:Arial'><b>Sample patient care programs</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Late test results</b></li>
    <li><b>Long wait times</b></li>
    <li><b>Missing prescriptions</b></li>
    <li><b>Argumentative staff</b></li>
    <li><b>Unhelpful advice</b></li>
    </ul>

In [ ]:
l = ['Late test results',
      'Long wait times',
      'Missing prescriptions',
      'Argumentative staff',
      'Unhelpful advice'
      ]


<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>7.3 Identify patient and program match</b></p>

<p style = 'font-size:16px;font-family:Arial'>Provide these results in real-time to call center agent dashboards, content management systems, or other marketing automation applications.</p>

In [ ]:
topics_list = l

d = widgets.Dropdown(options=topics_list)

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()
            local_df = vs.similarity_search(question = change['new']).similar_objects[['first_name', 'last_name','Transcript']].to_pandas()
            ipydisplay(local_df)
            text = " ".join(comment for comment in local_df['Transcript'])
            stopwords = set(STOPWORDS)
            wordcloud = WordCloud(stopwords = stopwords, width = 800, height=400, 
                                  max_words=100, min_word_length=1, 
                                  collocations = True, background_color = 'white').generate(text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='gaussian')
            plt.axis('off')
            plt.title(f'Word Cloud for Search Topic')
            plt.show()
            
d.observe(on_change)

print('Select Topic to Search:')
ipydisplay(d, output)


<hr>
<p style = 'font-size:24px;font-family:Arial'><b>Conclusion - Operationalizing AI-powered analytics</b></p>



<p style = 'font-size:16px;font-family:Arial'>The preceding demo showed two real-world applications of AI-powered analytics that leverages <b>private</b> data <b>securely</b>, to allow for deeper insight into unstructured data that can improve patient satisfaction.</p>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>8. Cleanup</b></p>



<p style = 'font-size:16px;font-family:Arial'>Call the destroy() method of the VS object to clean up the objects created during this demo.</p>

In [ ]:
vs.destroy()

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>